In [48]:
from dask_gateway import Gateway
from dask_gateway import GatewayCluster
from dask.distributed import Client

import socket
import pprint
import numpy

import os
import sys

import boto3

In [49]:
def get_s3_keys(bucket, s3_client, prefix = ''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    
    kwargs = {'Bucket': bucket}

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    done = False
    while not done:
        resp = s3_client.list_objects_v2(**kwargs)
        try:
            for obj in resp['Contents']:
                key = obj['Key']
                if key.startswith(prefix):
                    # print('key: ',key)
                    yield key
        except KeyError:
            print('Empty response from s3 for bucket %s with prefix %s'%(bucket,prefix))
            break

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break
            


In [50]:
gateway = Gateway()

In [51]:
gateway

Gateway<http://10.100.222.95:8000/services/dask-gateway>

In [52]:
options = gateway.cluster_options(); options

In [53]:
cluster = gateway.new_cluster(); cluster

In [54]:
clusters = gateway.list_clusters(); clusters

[ClusterReport<name=daskhub.651fbce5ad1c46f5b7284d524266edd3, status=RUNNING>]

In [8]:
# gateway_uri = "gateway://traefik-dhub-dask-gateway.daskhub:80/daskhub."+gateway.list_clusters()[0].name.split('.')[1]; gateway_uri

In [55]:
# client = cluster.get_client(gateway_uri);
client = cluster.get_client();

/home/jovyan/users_conda_envs/stare-1/lib/python3.7/site-packages/distributed/client.py:1136: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| numpy   | 1.19.2 | 1.19.1    | 1.19.1  |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [56]:
client

Client Scheduler: gateway://traefik-dhub-dask-gateway.daskhub:80/daskhub.651fbce5ad1c46f5b7284d524266edd3 Dashboard: /services/dask-gateway/clusters/daskhub.651fbce5ad1c46f5b7284d524266edd3/status,Cluster Workers: 1 Cores: 2 Memory: 4.29 GB


In [57]:
import dask.array as da
a = da.random.normal(size=(1000,1000),chunks=(50,50))
a.mean().compute()

-0.000898992625004862

In [58]:
client.nthreads()

{'tls://192.168.29.137:46465': 2}

In [59]:
# Dask helpers
def slam(client,action,data,partition_factor=1.5,dbg=0):
    np = sum(client.nthreads().values())
    if dbg>0: # a
        print('slam: np = %i'%np)
    shard_bounds = [int(i*len(data)/(1.0*partition_factor*np)) for i in range(int(partition_factor*np))] 
    if shard_bounds[-1] != len(data):
        if dbg>0: # b
            print('a sb[-1]: ',shard_bounds[-1],len(data))
        shard_bounds = shard_bounds + [len(data)]
        if dbg>0: # c
            print('sb: ',shard_bounds)
    data_shards = [data[shard_bounds[i]:shard_bounds[i+1]] for i in range(len(shard_bounds)-1)]
    if dbg>0: # d
        print('ds len:        ',len(data_shards))
        print('ds item len:   ',len(data_shards[0]))
        print('ds type:       ',type(data_shards[0]))
        try:
            print('ds dtype:      ',data_shards[0].dtype)
        except:
            pass
    big_future = client.scatter(data_shards)
    results    = client.map(action,big_future)
    return results


In [71]:
def action(data_shard):
    ret = []
    for d in data_shard:
        if False:
            ret.append((d,'touched',os.getcwd(),os.environ))
        if False:
            ret.append((d,'touched',os.getcwd(),os.listdir()))
        if False:
            ret.append((d,'touched',socket.gethostname()))
        if False:
            filename='.bashrc'
            with open(filename, 'r') as file:
                data = file.read()
            ret.append((d,'read .bashrc',data))
        if False:
            ret.append((d,os.environ['DASK_PARENT'],os.environ['DASK_GATEWAY_WORKER_NAME']))
            
        # Here's what we really need to work.
        if True:
            client = boto3.client('s3')
            keys = get_s3_keys('daskhub-data',client,prefix='')
            ret.append(d,list(keys)[0:10])
            
    return ret

In [72]:
if True:
    s3_client = boto3.client('s3')
    keys = get_s3_keys('daskhub-data',s3_client,prefix='')
    pprint.pprint(list(keys)[0:10])

['GESDISC/MERRA2/2019/MERRA2_400.tavg1_2d_slv_Nx.20191203.nc4',
 'GESDISC/MERRA2/2019/MERRA2_400.tavg1_2d_slv_Nx.20191222.nc4',
 'GESDISC/MERRA2/2019/MERRA2_400.tavg1_2d_slv_Nx.20191226.nc4',
 'MODAPS/CATALOGS/MOD05_L2/2019/356/catalogue.sqlite',
 'MODAPS/MOD05/MOD05_L2.A2019336.2315.061.2019337071952.hdf',
 'MODAPS/MOD05/MOD05_L2.A2019336.2315.061.2019337071952_stare.nc',
 'MODAPS/MOD05/MOD05_L2.A2019336.2320.061.2019337072008.hdf',
 'MODAPS/MOD05/MOD05_L2.A2019336.2320.061.2019337072008_stare.nc',
 'MODAPS/MOD05/MOD05_L2.A2019336.2325.061.2019337072403.hdf',
 'MODAPS/MOD05/MOD05_L2.A2019336.2325.061.2019337072403_stare.nc']


In [73]:
data = numpy.arange(10)

In [74]:
results = slam(client,action,data)

In [69]:
results

[<Future: finished, type: builtins.list, key: action-70e64675e77c04c887d1ffa2f815c9db>,
 <Future: finished, type: builtins.list, key: action-66bdd107bc6342af9f97bff89f027dc4>,
 <Future: finished, type: builtins.list, key: action-5647a27d47312b2b06c9665c8509dd26>]

In [75]:
client.gather(results)
# print(results[0].result()[0][2])

PermissionError: [Errno 13] Permission denied: '/var/run/secrets/eks.amazonaws.com/serviceaccount/token'

In [32]:
cluster.shutdown()

In [33]:
cluster.close()

In [34]:
gateway.list_clusters()

[]

In [ ]:
cluster = gateway.connect(clusters[0].name)

In [35]:
cluster.shutdown(); cluster.close()

In [10]:
gateway.close()

In [9]:
gateway.list_clusters()

[]

In [ ]:
gateway